In [1]:
import gensim
import pandas as pd
import numpy as np
from collections import defaultdict
from classify import experiment_train_test

import warnings
warnings.filterwarnings("ignore")

In [2]:
# load train data
df = pd.read_csv("../input/cleanseDAta.csv")

In [3]:
df.head()

,Unnamed: 0,review,sentiment
0,0,one reviewer mention watch oz episode hook rig...,positive
1,1,wonderful little production film technique una...,positive
2,2,think wonderful way spend time hot summer week...,positive
3,3,basically family little boy jake think zombie ...,negative
4,4,petter mattei love time money visually stunnin...,positive


In [4]:
# I couldn't figure out why it was working for pretrained data and not the one which I trained

import gensim.downloader as api

wv = api.load('glove-twitter-100')

In [7]:
def review_vec(review):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    count = 1
    for w in review:
        if w in wv:
            count += 1
            wv_res += wv[w]
    wv_res = wv_res / count
    return wv_res

In [9]:
df['review_'] = df['review'].apply(gensim.utils.simple_preprocess)

In [10]:
df.head()

,Unnamed: 0,review,sentiment,review_
0,0,one reviewer mention watch oz episode hook rig...,positive,"[one, reviewer, mention, watch, oz, episode, h..."
1,1,wonderful little production film technique una...,positive,"[wonderful, little, production, film, techniqu..."
2,2,think wonderful way spend time hot summer week...,positive,"[think, wonderful, way, spend, time, hot, summ..."
3,3,basically family little boy jake think zombie ...,negative,"[basically, family, little, boy, jake, think, ..."
4,4,petter mattei love time money visually stunnin...,positive,"[petter, mattei, love, time, money, visually, ..."


In [11]:
df['rvec'] = df['review_'].apply(review_vec)

In [12]:
df.head()

,Unnamed: 0,review,sentiment,review_,rvec
0,0,one reviewer mention watch oz episode hook rig...,positive,"[one, reviewer, mention, watch, oz, episode, h...","[0.1393422601256697, 0.047620350849770364, -0...."
1,1,wonderful little production film technique una...,positive,"[wonderful, little, production, film, techniqu...","[0.16146557441069967, -0.15833442986366295, 0...."
2,2,think wonderful way spend time hot summer week...,positive,"[think, wonderful, way, spend, time, hot, summ...","[-0.06501708436934721, 0.031096652948430607, 0..."
3,3,basically family little boy jake think zombie ...,negative,"[basically, family, little, boy, jake, think, ...","[0.1601498081526231, 0.07989171475526832, -0.0..."
4,4,petter mattei love time money visually stunnin...,positive,"[petter, mattei, love, time, money, visually, ...","[0.028575998506442244, 0.12349466740020684, 0...."


In [13]:
df['rvec'][7]

array([ 2.25504987e-02,  2.92397590e-01,  2.69427954e-01, -1.21774182e-01,
       -2.11877274e-02,  2.71736182e-01,  3.24455998e-01, -1.46037819e-01,
       -7.74033206e-02, -5.17772727e-02,  4.05507266e-02, -1.35997774e-01,
       -3.83205913e+00,  1.94357744e-02,  1.69612728e-01, -5.35069128e-02,
        1.11149842e-02, -1.28138077e-01, -2.72881832e-02,  4.56445495e-02,
       -5.00354068e-02,  1.49132959e-01, -1.10092996e-04, -8.50658185e-02,
       -1.62093172e-02, -1.15547504e-01, -9.59683646e-02, -1.04989315e-01,
        8.16304518e-02, -1.19000680e-01, -9.09344825e-02, -4.80287292e-02,
       -3.23294723e-01,  1.05838634e-01,  2.86735185e-01,  6.01419534e-02,
        2.84420776e-01, -1.29222483e-01, -1.65474371e-01, -1.04291628e-01,
       -5.15806367e-01, -9.38991403e-02, -6.67676801e-02,  6.57104982e-02,
        3.50996817e-01,  1.10478136e-01,  1.00920637e-01, -1.43978457e-01,
       -6.83072404e-03, -2.15129119e-02,  2.30571000e-02,  1.03445364e-01,
       -1.59233463e-01,  

In [14]:
input_path = "../input/"
percentToTrain = 70
numberToTrain = int(len(df)*(percentToTrain/100));
trainingData = df.head(int(len(df)*(percentToTrain/100)))

percentToTest = 100 - percentToTrain
testData = df.tail(int(len(df) * (percentToTest/100)))

In [15]:
trainingData.shape

(35000, 5)

In [16]:
testData.shape

(15000, 5)

In [17]:
X_train = trainingData['rvec'].to_list()
y_train = trainingData['sentiment'].to_list()

In [18]:
X_test = testData['rvec'].to_list()
y_test = testData['sentiment'].to_list()

In [19]:
# train / test

# removed naive bayes because it doesn't work for negative values
models = ['svm', 'lr']
results = defaultdict()

for model in models:
  results[model] = experiment_train_test(X_train=X_train,
                                         y_train=y_train, 
                                         X_test=X_test, 
                                         y_test=y_test, 
                                         classifier=model,
                                         verbose=True)
results

LinearSVC()
              precision    recall  f1-score   support

    negative      0.733     0.713     0.723      7490
    positive      0.721     0.741     0.731      7510

    accuracy                          0.727     15000
   macro avg      0.727     0.727     0.727     15000
weighted avg      0.727     0.727     0.727     15000

LogisticRegression(solver='liblinear')
              precision    recall  f1-score   support

    negative      0.732     0.716     0.724      7490
    positive      0.723     0.738     0.731      7510

    accuracy                          0.727     15000
   macro avg      0.727     0.727     0.727     15000
weighted avg      0.727     0.727     0.727     15000



defaultdict(None,
            {'svm': {'model': LinearSVC(),
              'score': 0.7269417949130037,
              'predictions': array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
                     'negative'], dtype='<U8')},
             'lr': {'model': LogisticRegression(solver='liblinear'),
              'score': 0.727226949698218,
              'predictions': array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
                     'negative'], dtype='<U8')}})

In [20]:
# f-1 scores
for k,v in results.items():
  print(f"{k}:\t{v['score']:.3f}")

svm:	0.727
lr:	0.727
